In [14]:
import pandas as pd
import os as os
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [15]:
SERVICE_ACCOUNT_FILE = r"X:/SGE/GABINETE/BI PAINEL DE CONFORMIDADE/iconic-apricot-248419-250fa518f7ae.json"
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SHEET_ID = "1_teMusgzqisvbbL3TOONcjJSBibTTae5AIKp-oeceQg"
RANGE = "A1:Q1000"  # Ajuste o intervalo conforme necessário
credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)

FileNotFoundError: [Errno 2] No such file or directory: 'X:/SGE/GABINETE/BI PAINEL DE CONFORMIDADE/iconic-apricot-248419-250fa518f7ae.json'

In [6]:
https://docs.google.com/spreadsheets/d/1_teMusgzqisvbbL3TOONcjJSBibTTae5AIKp-oeceQg/edit?gid=656302965#gid=656302965

['https://www.googleapis.com/auth/spreadsheets.readonly']
